In [ ]:
!nvidia-smi

Fri Mar 14 12:09:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip uninstall paddlepaddle paddlepaddle-gpu -y

In [ ]:
!pip install paddlepaddle-gpu==2.6.0.post120 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install --upgrade paddlenlp

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.0/788.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:0

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
data_path = "/content/drive/MyDrive/Colab Notebooks/DL/"  # this is your drive

Mounted at /content/drive/


In [ ]:
import json
import re

# 假设你的JSON文件路径
json_file_path = data_path + 'train_data/ICD-Coding-train.json'

# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 存放最终结果
result = []

# 遍历每个病例
for record in data:
    #病例标识
    bingli = record.get('病案标识', '')

    #主要诊断编码和其他诊断编码
    main_code = record.get('主要诊断编码', '')
    other_codes = record.get('其他诊断编码', '')


    # 主要诊断字段组合
    zhusu = record.get('主诉', '')
    xianbingshi = record.get('现病史', '')
    ruyuanqingkuang = record.get('入院情况', '')

    # 入院诊断处理
    ruyuan_zhenduan = record.get('入院诊断', '')
    # 提取 "1." 到 "2." 之间的内容
    ryzd_1 = ''
    match_1 = re.search(r'1\.(.*?)(2\.|$)', ruyuan_zhenduan, re.S)
    if match_1:
        ryzd_1 = match_1.group(1).strip()

    # 诊疗经过和出院情况
    zhenliao_guocheng = record.get('诊疗经过', '')
    chuyuan_qingkuang = record.get('出院情况', '')

    # 合并主要诊断文本
    main_text = f"{zhusu} {xianbingshi} {ruyuanqingkuang} {ryzd_1} {zhenliao_guocheng} {chuyuan_qingkuang}".strip()

    # 其他诊断字段组合
    jiwangshi = record.get('既往史', '')
    geren_shi = record.get('个人史', '')
    hunyin_shi = record.get('婚姻史', '')
    jiazu_shi = record.get('家族史', '')

    # 提取入院诊断 "2." 后的部分
    ryzd_2plus = ''
    match_2 = re.search(r'2\.(.*)', ruyuan_zhenduan, re.S)
    if match_2:
        ryzd_2plus = match_2.group(1).strip()

    # 出院医嘱
    chuyuan_yizhu = record.get('出院医嘱', '')

    # 合并其他诊断文本
    other_text = f"{jiwangshi} {geren_shi} {hunyin_shi} {jiazu_shi} {ryzd_2plus} {zhenliao_guocheng} {chuyuan_yizhu}".strip()

    # 拼装结果
    result.append({
        '病案标识': bingli,
        '主要诊断病例文本': main_text,
        '其他诊断病例文本': other_text,
        '主要诊断编码': main_code,
        '其他诊断编码': other_codes

    })

# 打印或者保存结果
output_file_path = data_path + 'combine_text.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print(f"处理完成，结果已保存到 {output_file_path}")


处理完成，结果已保存到 /content/drive/MyDrive/Colab Notebooks/DL/combine_text.json


In [ ]:
import paddle
from paddlenlp import Taskflow

print("PaddlePaddle版本：", paddle.__version__)
print("是否使用GPU：", paddle.device.is_compiled_with_cuda())
print("当前设备：", paddle.device.get_device())

/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


PaddlePaddle版本： 2.6.0
是否使用GPU： True
当前设备： gpu:0


In [ ]:
import json
import time
import paddle
from paddlenlp import Taskflow

# 定义Schema
entity_schema = [
    '疾病', '症状', '检查项目', '治疗方法',
    '身体部位', '发病时间', '药物', '手术'
]

relation_schema = {
    '疾病': ['相关症状', '检查方法', '治疗方法'],
    '症状': ['诱发因素', '缓解因素', '伴随症状'],
    '治疗方法': ['治疗疾病', '用药剂量']
}

# 联合实体+关系抽取Schema
full_schema = entity_schema + list(relation_schema.keys())

# 加载医疗模型（自动加速）
try:
    ie_medical = Taskflow("information_extraction",
                          schema=full_schema,
                          task_path="uie-medical-base",
                          device_id=0)
except:
    ie_medical = Taskflow("information_extraction",
                          schema=full_schema,
                          device_id=0)

# 去重函数
def deduplicate_entities_and_relations(results):
    deduped_results = []
    for item in results:
        deduped_item = {}
        for entity_type, entities in item.items():
            seen = set()
            deduped_entities = []
            for entity in entities:
                text = entity['text']
                if text not in seen:
                    seen.add(text)
                    if 'relations' in entity:
                        relations = {}
                        for rel_type, rel_list in entity['relations'].items():
                            rel_seen = set()
                            dedup_rel_list = []
                            for rel in rel_list:
                                rel_text = rel['text']
                                if rel_text not in rel_seen:
                                    rel_seen.add(rel_text)
                                    dedup_rel_list.append(rel)
                            relations[rel_type] = dedup_rel_list
                        entity['relations'] = relations
                    deduped_entities.append(entity)
            deduped_item[entity_type] = deduped_entities
        deduped_results.append(deduped_item)
    return deduped_results


def simplify_entity_text(data):
    simplified_data = []
    for item in data:
        simplified_item = {}
        for entity_type, entities in item.items():
            # 只提取 text 字段，拼接成字符串
            texts = [entity['text'] for entity in entities]
            simplified_item[entity_type] = '，'.join(texts)
        simplified_data.append(simplified_item)
    return simplified_data

# 开始计时（整体运行时间）
start_time = time.time()

# 加载result JSON文件（假设文件叫 combine_text.json）
print("⏳ 开始加载 JSON 文件...")
load_start = time.time()
with open(data_path + 'train_data/combine_text.json', 'r', encoding='utf-8') as f:
    result_data = json.load(f)
load_end = time.time()
print(f"✅ JSON 文件加载完成，耗时: {load_end - load_start:.2f} 秒")

# 存储最终结果
final_output = []

# 遍历处理每条记录
print(f"\n📊 开始处理 {len(result_data)} 条记录...")
for idx, item in enumerate(result_data):
    loop_start = time.time()

    # 病例标识
    bingli = item.get('病案标识', '')

    # 主要诊断编码和其他诊断编码
    main_code = item.get('主要诊断编码', '')
    other_codes = item.get('其他诊断编码', '')

    main_text = item.get("主要诊断病例文本", "")
    other_text = item.get("其他诊断病例文本", "")

    # 对主要诊断文本执行信息抽取
    print(f"\n🚀 正在抽取第 {idx+1} 条记录的主要诊断病例文本...")
    main_extract_start = time.time()
    main_results = ie_medical(main_text)
    deduped_main = deduplicate_entities_and_relations(main_results)
    simplified_main = simplify_entity_text(deduped_main)
    main_extract_end = time.time()
    print(f"✅ 主要诊断抽取完成，耗时: {main_extract_end - main_extract_start:.2f} 秒")

    # 对其他诊断文本执行信息抽取
    print(f"\n🚀 正在抽取第 {idx+1} 条记录的其他诊断病例文本...")
    other_extract_start = time.time()
    other_results = ie_medical(other_text)
    deduped_other = deduplicate_entities_and_relations(other_results)
    simplified_other = simplify_entity_text(deduped_other)
    other_extract_end = time.time()
    print(f"✅ 其他诊断抽取完成，耗时: {other_extract_end - other_extract_start:.2f} 秒")

    # 组合结果
    final_output.append({
        '病案标识': bingli,
        "抽取后主要诊断病例文本": simplified_main,
        "抽取后其他诊断病例文本": simplified_other,
        '主要诊断编码': main_code,
        '其他诊断编码': other_codes
    })

    loop_end = time.time()
    print(f"📌 第 {idx+1}/{len(result_data)} 条记录处理完成，总耗时: {loop_end - loop_start:.2f} 秒")

# 保存结果到文件
print("\n⏳ 开始保存结果到文件...")
save_start = time.time()
output_file = data_path + "train_data/extracted_text.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(final_output, f, ensure_ascii=False, indent=4)
save_end = time.time()
print(f"✅ 结果保存完成，耗时: {save_end - save_start:.2f} 秒")

# 结束计时（整体运行时间）
end_time = time.time()
total_time = end_time - start_time
print(f"\n🎉 全部处理完成，总耗时: {total_time:.2f} 秒")
print(f"结果已保存到：{output_file}")

[2025-03-14 13:24:09,045] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'uie-medical-base'.


流式输出内容被截断，只能显示最后 5000 行内容。
🚀 正在抽取第 87 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.78 秒
📌 第 87/800 条记录处理完成，总耗时: 1.95 秒

🚀 正在抽取第 88 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 1.29 秒

🚀 正在抽取第 88 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.79 秒
📌 第 88/800 条记录处理完成，总耗时: 2.09 秒

🚀 正在抽取第 89 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 2.06 秒

🚀 正在抽取第 89 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 1.57 秒
📌 第 89/800 条记录处理完成，总耗时: 3.63 秒

🚀 正在抽取第 90 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 2.92 秒

🚀 正在抽取第 90 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 2.36 秒
📌 第 90/800 条记录处理完成，总耗时: 5.28 秒

🚀 正在抽取第 91 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 1.16 秒

🚀 正在抽取第 91 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.81 秒
📌 第 91/800 条记录处理完成，总耗时: 1.97 秒

🚀 正在抽取第 92 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 1.61 秒

🚀 正在抽取第 92 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 1.16 秒
📌 第 92/800 条记录处理完成，总耗时: 2.76 秒

🚀 正在抽取第 93 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.81 秒

🚀 正在抽取第 93 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.79 秒
📌 第 93/800 条记录处理完成，总耗时: 1.60 秒

🚀 正在抽取第 94 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 1.64 秒

🚀 正在抽取第 94 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 1.13 秒
📌 第 94/